In [7]:
'''Main libraries'''
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

'''Data Prep'''
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

'''Algorithms'''
# import xgboost as xgb

'Algorithms'

In [6]:
# Import CSV 
df_sales = pd.read_csv('/Users/mbp/Documents/ML-Projects/forecast_product_sales/data/product_sales_dataset_weekly.csv')
# Preview dataset
df_sales.head(5)

,Scode,Pcode,Price,Wk0,Wk1,Wk2,Wk3,Wk4,Wk5,Wk6,...,Wk96,Wk97,Wk98,Wk99,Wk100,Wk101,Wk102,Wk103,Unnamed: 107,94
0,Store1,SKU1,$24.00,23,48,47,42,22,29,47,...,33,40,125,133,229,381,21,31,NaN,NaN
1,Store1,SKU2,$63.00,23,22,25,40,32,26,30,...,25,50,75,95,231,590,25,33,NaN,NaN
2,Store1,SKU3,$93.00,7,11,8,9,10,8,7,...,13,30,67,188,267,330,35,33,NaN,NaN
3,Store1,SKU4,$48.00,50,31,50,33,26,27,42,...,26,23,92,126,205,357,30,34,NaN,NaN
4,Store1,SKU5,$62.00,8,5,13,11,6,7,9,...,15,21,93,243,710,1872,33,10,NaN,NaN


In [8]:
# reshape dataframe: From wide to long dataframe 
def wide_to_long(df: DataFrame) -> DataFrame:
    # define variables
    id_vars = ['Scode', 'Pcode', 'Price']
    value_var = ['Wk'+str(i) for i in range(0,104)]
    var_name = 'weekly'
    value_name = 'units'

    rename_columns = ['store', 'item', 'price', 'weekly', 'units']
    df = df.melt(id_vars=id_vars,
                value_vars=value_var,
                var_name=var_name,
                value_name=value_name)

    df.columns = rename_columns
    return df

In [11]:
df = wide_to_long(df_sales)
df.

,store,item,price,weekly,units
0,Store1,SKU1,$24.00,Wk0,23
6000,Store1,SKU1,$24.00,Wk1,48
60000,Store1,SKU1,$24.00,Wk10,27
600000,Store1,SKU1,$24.00,Wk100,229
606000,Store1,SKU1,$24.00,Wk101,381
...,...,...,...,...,...
575179,Store5,SKU999,$25.00,Wk95,41
581179,Store5,SKU999,$25.00,Wk96,42
587179,Store5,SKU999,$25.00,Wk97,43
593179,Store5,SKU999,$25.00,Wk98,70
